In [1]:
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout


In [2]:
%matplotlib inline
# import sys
# reload(sys)
# sys.setdefaultencoding('utf8')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.stats as stats
import plotly
plotly.tools.set_credentials_file(username='prdpram08', api_key='vahwmX6mq11V2fHIeQi6')
import plotly.plotly as py
from plotly.graph_objs import *
import string
import plotly.graph_objs as go
import cufflinks as cf
import xlrd
# from sklearn.metrics import classification_report, confusion_matrix
# from sklearn.cross_validation import train_test_split
# from sklearn.linear_model import LassoLarsCV
# from sklearn.svm import SVC, LinearSVC
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.linear_model import Perceptron
# from sklearn.linear_model import SGDClassifier
# from sklearn.tree import DecisionTreeClassifier
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.max_colwidth = 500
#df= pd.read_excel(open('C:\Users\pradeep.ram\Downloads\propsalanalysis\pr.xlsx','rb'), sheetname='Sheet1', encoding='ISO-8859-1')
#df= pd.read_excel(open('C:\Users\pradeep.ram\Downloads\propsalanalysis\pr.xlsx','rb'), sheetname='Sheet1', encoding='UTF-8')


In [3]:
import xlrd
file = 'data4.xlsx'
xls = xlrd.open_workbook(file)
l1=xls.sheet_names()
dfs=[]
for sheet_name in l1:
    df= pd.read_excel(open(file,'rb'), sheetname=sheet_name)
    dt=df.iloc[5,3]
    df = df.iloc[:,1:3]
    df = df.iloc[10:]
    df.columns =['Title','Description']
    df = df.reset_index(drop=True)
    for index, row in df.iterrows():
        if pd.isnull(row["Title"])==True:
            p=index
            break
    
    df = df.iloc[p+2:]    
    df.dropna(how='all',inplace=True)
    df.drop(df.tail(1).index,inplace=True)
    df.Description = df.Description.replace(np.nan, 'project', regex=True)
    df.Description=df.Description.astype(str)
    df.Title = df.Title.fillna(method='ffill')
    df["Count"]=1
    j=1
    for i in range(0,len(df)-1):
        if df.iloc[i,0]!=df.iloc[i+1,0]:
            df.iloc[i,2]=j
            j=j+1
        else:   
            df.iloc[i,2]=j
    dfg=df.groupby(['Count'])['Description'].apply(lambda x: ",".join(x)).reset_index()
    df = df.drop_duplicates(subset=['Count'])
    df.columns=["Title","Desc","Count"]
    df=pd.merge(df, dfg, on='Count')
    df.drop(['Desc','Count'], axis=1, inplace=True)
    df.iloc[0,1]='project'
    h=['Bidding Type','Client Address','Deadline','Description','Estimated Cost','Financier','Latest Update','Organisation','Project Location','Related Documents :','TI Ref. No.','Tender Notice No.']
    for index, row in df.iterrows():
        if (row["Description"]=="project") & (row["Title"] not in h):
            row["Description"]=row["Title"]
            row["Title"]="Project Title"
    df = df.set_index([df['Title'].eq('Project Title').cumsum(), 'Title'])['Description'].unstack().reset_index(drop=True)   
    df['Date']=dt
    dfs.append(df)
    df=[]

data = pd.concat(dfs,axis=0)
data = data.reset_index(drop=True)

In [4]:
data1=data[['Bidding Type',"Project Title","Description","Organisation","Project Location","Client Address",'Related Documents :',"Deadline","Estimated Cost",'Date']]
data1.head(10)

Title,Bidding Type,Project Title,Description,Organisation,Project Location,Client Address,Related Documents :,Deadline,Estimated Cost,Date
0,Domestic Competitive Bidding,1.India - Rate Contract For Digital Signature Certificate Class Iii Only Sign Single Token With 2 Years Valid,"Tenders are invited for Rate contract for digital signature certificate class iii only sign single token with 2 years valid,Type: Open,Eligibility Criteria : -",EAST COAST RAILWAY,India,"Nandan Kanan Rd, Chandrasekharpur, Bhubaneshwar - 751017 , Orissa",Original Document : Document,2018-08-06 00:00:00,None,"Thursday, Jul 12, 2018"
1,Domestic Competitive Bidding,"2.India - Preparation Of Gis Based Master Plan For Karaikal, Mane & Yanam Planning Areas Of U.t. Of Puducherry","Tenders are invited for Preparation of GIS based Master Plan for Karaikal, Mane & Yanam Planning Areas of U.T. of Puducherry",TOWN AND COUNTRY PLANNING DEPARTMENT,India,"Jawahar Nagar Main Road, Jawahar Nagar, Boomianpet, Puducherry, 605005,Tel: +91-0413-2201952,Email_id: ctptcp.pon@nic.in",Original Document,2018-08-03 00:00:00,None,"Thursday, Jul 12, 2018"
2,Domestic Competitive Bidding,3.India - P/e New Street Light Point Of L.e.d From Old Octroi Post To City At Sarainaga Raod.,"Tenders are invited for P/e new street light point of l.e.d from old octroi post to city at sarainaga raod.,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No Eligibility Criteria : Please refer Tender documents. Document cost : INR 100 EMD value : INR 6000 Document Purcha...",DEPARTMENT OF LOCAL GOVERNMENT,India,"Director - Local Government||Regional Deputy Director - Bathinda||Nagar Panchayat - Bariwala, Np Bariwala",Original Document : Bid Document,2018-07-25 00:00:00,"INR, 300000","Thursday, Jul 12, 2018"
3,Domestic Competitive Bidding,4.India - Supply Of Goods For Street Light Maintance,"Tenders are invited for Supply of goods for street light maintance,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No Eligibility Criteria : As per DNIT EMD value : INR 3000 Document Purchase Start date : 12 Jul 2018 Document Purchase End date : 06 Aug 2018 Opening date : ...",DEPARTMENT OF LOCAL GOVERNMENT,India,"Director - Local Government||Regional Deputy Director - Patiala||Nagar Panchayat - Munak, Np Moonak",Original Document : Bid Document,2018-08-06 00:00:00,"INR, 150000","Thursday, Jul 12, 2018"
4,Domestic Competitive Bidding,5.India - Supply Of Tree Guard,"Tenders are invited for Supply of tree guard,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No Eligibility Criteria : As per DNIT EMD value : INR 4000 Document Purchase Start date : 12 Jul 2018 Document Purchase End date : 06 Aug 2018 Opening date : 06 Aug 2018",DEPARTMENT OF LOCAL GOVERNMENT,India,"Director - Local Government||Regional Deputy Director - Patiala||Nagar Panchayat - Munak, Np Moonak",Original Document : Bid Document,2018-08-06 00:00:00,"INR, 200000","Thursday, Jul 12, 2018"
5,Domestic Competitive Bidding,6.India - Consultancy Services For Preparation Of Feasibility Study And Detailed Project Report In The State Of Punjab For Improvement Of Existing Road Geometric Design Safety Measures And Other Related Works F,"Tenders are invited for Consultancy services for preparation of feasibility study and detailed project report in the state of punjab for improvement of existing road geometric 

In [5]:
data1["DescriptionA"]=data1["Description"]

In [6]:
data1['DescriptionA'] = data1['DescriptionA'].str.replace(',', ' ')
# data1['DescriptionA'] = data1['DescriptionA'].str.replace('.', ' ')
data1['DescriptionA'] = data1['DescriptionA'].str.replace('-', ' ')
data1['DescriptionA'] = data1['DescriptionA'].str.replace('(', ' ')
data1['DescriptionA'] = data1['DescriptionA'].str.replace(')', ' ')
data1['DescriptionA'] = data1['DescriptionA'].str.replace(':', ' ')
data1['DescriptionA'] = data1['DescriptionA'].str.replace(';', ' ')
data1['DescriptionA'] = data1['DescriptionA'].str.replace('"', ' ')
data1['DescriptionA'] = data1['DescriptionA'].str.replace('/', ' ')
data1['DescriptionA'] = data1['DescriptionA'].str.replace('%', ' ')
data1['DescriptionA'] = data1['DescriptionA'].str.replace('&', ' ')
data1['DescriptionA'] = data1['DescriptionA'].str.replace("'", ' ')
data1['DescriptionA'] = data1['DescriptionA'].str.replace(".", ' ')
data1['DescriptionA'] = data1['DescriptionA'].str.lower()
data1.head()

Title,Bidding Type,Project Title,Description,Organisation,Project Location,Client Address,Related Documents :,Deadline,Estimated Cost,Date,DescriptionA
0,Domestic Competitive Bidding,1.India - Rate Contract For Digital Signature Certificate Class Iii Only Sign Single Token With 2 Years Valid,"Tenders are invited for Rate contract for digital signature certificate class iii only sign single token with 2 years valid,Type: Open,Eligibility Criteria : -",EAST COAST RAILWAY,India,"Nandan Kanan Rd, Chandrasekharpur, Bhubaneshwar - 751017 , Orissa",Original Document : Document,2018-08-06 00:00:00,None,"Thursday, Jul 12, 2018",tenders are invited for rate contract for digital signature certificate class iii only sign single token with 2 years valid type open eligibility criteria
1,Domestic Competitive Bidding,"2.India - Preparation Of Gis Based Master Plan For Karaikal, Mane & Yanam Planning Areas Of U.t. Of Puducherry","Tenders are invited for Preparation of GIS based Master Plan for Karaikal, Mane & Yanam Planning Areas of U.T. of Puducherry",TOWN AND COUNTRY PLANNING DEPARTMENT,India,"Jawahar Nagar Main Road, Jawahar Nagar, Boomianpet, Puducherry, 605005,Tel: +91-0413-2201952,Email_id: ctptcp.pon@nic.in",Original Document,2018-08-03 00:00:00,None,"Thursday, Jul 12, 2018",tenders are invited for preparation of gis based master plan for karaikal mane yanam planning areas of u t of puducherry
2,Domestic Competitive Bidding,3.India - P/e New Street Light Point Of L.e.d From Old Octroi Post To City At Sarainaga Raod.,"Tenders are invited for P/e new street light point of l.e.d from old octroi post to city at sarainaga raod.,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No Eligibility Criteria : Please refer Tender documents. Document cost : INR 100 EMD value : INR 6000 Document Purcha...",DEPARTMENT OF LOCAL GOVERNMENT,India,"Director - Local Government||Regional Deputy Director - Bathinda||Nagar Panchayat - Bariwala, Np Bariwala",Original Document : Bid Document,2018-07-25 00:00:00,"INR, 300000","Thursday, Jul 12, 2018",tenders are invited for p e new street light point of l e d from old octroi post to city at sarainaga raod tender type open tender form of contract percentage tender category works no of covers 2 general technical evaluation allowed no itemwise technical evaluation allowed no payment mode online is multi currency allowed for boq no is multi currency allowed for fee no eligibility criteria please refer tender documents document cost inr 100 emd value inr 6000 document purcha...
3,Domestic Competitive Bidding,4.India - Supply Of Goods For Street Light Maintance,"Tenders are invited for Supply of goods for street light maintance,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No Eligibility Criteria : As per DNIT EMD value : INR 3000 Document Purchase Start date : 12 Jul 2018 Document Purchase End date : 06 Aug 2018 Opening date : ...",DEPARTMENT OF LOCAL GOVERNMENT,India,"Director - Local Government||Regional Deputy Director - Patiala||Nagar Panchayat - Munak, Np Moonak",Original Document : Bid Document,2018-08-06 00:00:00,"INR, 150000","Thursday, Jul 12, 2018",tenders are invited for supply of goods for street light maintance tender type open tender form of contract percentage tender category works no of covers 2 general technical evaluation allowed no itemwise technical evaluation allowed no payment mode online is multi currency allowed for boq no is multi currency allowed for fee no eligibility criteria as per dnit emd value inr 3000 document purchase start date 12 jul 2018 docum

In [7]:
data1.dtypes

Title
Bidding Type           object
Project Title          object
Description            object
Organisation           object
Project Location       object
Client Address         object
Related Documents :    object
Deadline               object
Estimated Cost         object
Date                   object
DescriptionA           object
dtype: object

In [8]:
import nltk
# nltk.download('wordnet')
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

# def lemmatize_text(text):
#     return [lemmatizer.lemmatize(w,'v') for w in w_tokenizer.tokenize(text)]

from nltk.stem import PorterStemmer
porter = PorterStemmer()
def stem_text(text):
    return [porter.stem(w) for w in w_tokenizer.tokenize(text.decode('utf-8'))]
# i1['text_lemmatized'] = i1.Title.apply(lemmatize_text)
#df['text_lemmatized2'] = df.Title.apply(lemmatize_text)
data1['text_stem'] = data1["DescriptionA"].apply(stem_text)
data1.head()

Title,Bidding Type,Project Title,Description,Organisation,Project Location,Client Address,Related Documents :,Deadline,Estimated Cost,Date,DescriptionA,text_stem
0,Domestic Competitive Bidding,1.India - Rate Contract For Digital Signature Certificate Class Iii Only Sign Single Token With 2 Years Valid,"Tenders are invited for Rate contract for digital signature certificate class iii only sign single token with 2 years valid,Type: Open,Eligibility Criteria : -",EAST COAST RAILWAY,India,"Nandan Kanan Rd, Chandrasekharpur, Bhubaneshwar - 751017 , Orissa",Original Document : Document,2018-08-06 00:00:00,None,"Thursday, Jul 12, 2018",tenders are invited for rate contract for digital signature certificate class iii only sign single token with 2 years valid type open eligibility criteria,"[tender, are, invit, for, rate, contract, for, digit, signatur, certif, class, iii, onli, sign, singl, token, with, 2, year, valid, type, open, elig, criteria]"
1,Domestic Competitive Bidding,"2.India - Preparation Of Gis Based Master Plan For Karaikal, Mane & Yanam Planning Areas Of U.t. Of Puducherry","Tenders are invited for Preparation of GIS based Master Plan for Karaikal, Mane & Yanam Planning Areas of U.T. of Puducherry",TOWN AND COUNTRY PLANNING DEPARTMENT,India,"Jawahar Nagar Main Road, Jawahar Nagar, Boomianpet, Puducherry, 605005,Tel: +91-0413-2201952,Email_id: ctptcp.pon@nic.in",Original Document,2018-08-03 00:00:00,None,"Thursday, Jul 12, 2018",tenders are invited for preparation of gis based master plan for karaikal mane yanam planning areas of u t of puducherry,"[tender, are, invit, for, prepar, of, gi, base, master, plan, for, karaik, mane, yanam, plan, area, of, u, t, of, puducherri]"
2,Domestic Competitive Bidding,3.India - P/e New Street Light Point Of L.e.d From Old Octroi Post To City At Sarainaga Raod.,"Tenders are invited for P/e new street light point of l.e.d from old octroi post to city at sarainaga raod.,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No Eligibility Criteria : Please refer Tender documents. Document cost : INR 100 EMD value : INR 6000 Document Purcha...",DEPARTMENT OF LOCAL GOVERNMENT,India,"Director - Local Government||Regional Deputy Director - Bathinda||Nagar Panchayat - Bariwala, Np Bariwala",Original Document : Bid Document,2018-07-25 00:00:00,"INR, 300000","Thursday, Jul 12, 2018",tenders are invited for p e new street light point of l e d from old octroi post to city at sarainaga raod tender type open tender form of contract percentage tender category works no of covers 2 general technical evaluation allowed no itemwise technical evaluation allowed no payment mode online is multi currency allowed for boq no is multi currency allowed for fee no eligibility criteria please refer tender documents document cost inr 100 emd value inr 6000 document purcha...,"[tender, are, invit, for, p, e, new, street, light, point, of, l, e, d, from, old, octroi, post, to, citi, at, sarainaga, raod, tender, type, open, tender, form, of, contract, percentag, tender, categori, work, no, of, cover, 2, gener, technic, evalu, allow, no, itemwis, technic, evalu, allow, no, payment, mode, onlin, is, multi, currenc, allow, for, boq, no, is, multi, currenc, allow, for, fee, no, elig, criteria, pleas, refer, tender, document, document, cost, inr, 100, emd, valu, inr, 600..."
3,Domestic Competitive Bidding,4.India - Supply Of Goods For Street Light Maintance,"Tenders are invited for Supply of goods for street light maintance,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No Eligibility Criter

In [9]:
# import nltk
# # nltk.download('stopwords')
# from nltk.corpus import stopwords
# stop = stopwords.words('english')
# data1['text_stem']=data1['text_stem'].apply(lambda x: [item for item in x if item not in stop])
# data1.head()

In [10]:
# data1['text']=data1.text_new.apply(lambda x: ' '.join(x))

In [11]:
data1["BU"]=" "

In [12]:
enggword=['detail','prepar','road']
enggword1=['dpr','flyover']
enggword2=['dpr','by-pass']
enggword3=['locat','survey','digit','model']
enggword4=['project','manag','consult','hospit']
enggword5=['consult','road']
enggword6=['detail','report','bridg']
enggword7=['dpr','bridg']
enggword8=['dpr','road']
enggword9=['detail','report','flyover']
enggword10=['consult','structur']
enggword11=['consult','build']
enggword12=['consult','hous']
enggword13=['detail','report','survey']
enggword14=['develop', 'multimod', 'transit','hub']
enggword15=['detail','report','bu']
enggword16=['detail','prepar','lane']
enggword17=['consult','station']
enggword18=['consult','lane']
enggword19=['dpr','build']
enggword20=['prepar','draw','survey']
enggword21=['consult','architectur']
enggword22=['consult', 'prepar', 'zonal', 'develop', 'plan']
enggword23=['consult','landscap']
enggword24=['consult','packag']
enggword25=['consult','highway']
enggword26=['consult','bridg']
enggword27=["feasibl","studi","road"]
enggword28=['consult','terminu']
enggword29=['feasibl', 'studi','lane']
enggword30=['design', 'draw','road']
enggword31=['design', 'draw','lane']
enggword32=["studi","lane"]
enggword33=["consult","km"]
enggword34=["preliminari","km"]
enggword35=['consult','stand','bu']
enggword36=['consult','complex']
enggword37=['consult','develop','infrastructur']
enggword38=['consult','pmay']
enggword39=['consult','expressway']
enggword40=["studi","road"]
enggword41=["detail","report","complex"]
enggword42=["geolog","investig"]
enggword43=["consult","route"]
enggword44=["dpr","bu","stand"]
enggword45=["geotechn","studi"]
enggword46=["consult","bypass"]
enggword47=["geotechn","investig"]
enggword48=["topograph","survey"]
enggword49=["studi","build"]
railword=['detail','report','rail']
railword2=['detail','report','railway']
railword3=['consult','rail']
railword4=['consult','railway']
railword5=["studi","railway"]
railword6=["preliminari","railway"]
railword7=["studi","rail"]
railword8=["pmc","rail"]
railword9=["pmc","railway"]
railword10=['consult','side']
railword11=["design","draw","bridg"]
railword12=["estim","bridg"]
railword13=["consult","stn"]
railword14=["gad",'railway']
railword15=["plan",'railway']
bdword4=['consult','smart','citi']
# ttsword1=['detail','consult','smart','citi']
itsword1=['consult', 'automat', 'traffic','control']
itsword2=['master','system', 'integr']
itsword3=['integr','traffic']
itsword4=['traffic', 'signal']
itsword5=['instal', 'traffic']
itsword6=['integr','transport','system']
itsword7=['intellig','transport']
itsword8=['si','smart','citi']
itsword9=['traffic', 'control']
itsword10=['traffic', 'manag','system']
itsword=['repair','traffic','signal']
itword1=['ticket','bu','platform']
itword2=['implement','control','centr']
itword3=["vehicl","track"]
ttsword4=['consult','command','centr']
# ttsword6=['consult','smart','citi']
ttsword6=['consult','transport','implement']
# itword5=['system','integr']
ttsword5=['system','integr','consult','traffic']
itword6=['ict','master','plan']
ttsword7=['consult','cctv']
ttsword8=['consult','it']
itword8=['procur','gps']
itword9=['track','gps']
itword10=['intellig','transit']
itword11=['fare','system','collect']
erpword1=['erp','consult']
tpword1=['consult','transport']
tpword2=['consult','traffic']
tpword3=['consult','freight']
tpword4=['safeti','consult']
tpword5=['consult','multimod']
tpword6=["consult", "master","plan"]
tpword7=["consult", "mobil","plan"]
tpword8=["consult", "pedestrian"]
tpword9=["transport", "plan"]
tpword10=["urban", "plan"]
tpword11=["town", "plan"]
tpword12=["provis","transport","servic"]
tpword13=["region","plan"]
tpword14=["develop","road","plan"]
tpword15=["traffic","plan"]
tpword16=["develop","plan","prepar"]
tpword17=["integr","plan","develop"]
tpword18=["dpr","pedestrian"]
tpword19=["transport","studi"]
tpword20=["transport","document"]
tpword21=["traffic","impact"]
tpword22=['detail','report','freight']
tpword23=['master','plan']
tpword24=['road','safeti','audit']
tpword25=['socio','econom','impact']
tpword26=['transit','servic']
tpword27=['detail','report','town']
tpword28=['land','plan']
bpword1=["transact", "advisori"]
bpword2=["consult","car" "park"]
bpword3=["transact", "advisor"]
bpword4=["studi","car" "park"]
bpword5=["preliminari","car" "park"]
bpword6=["consult","ppp"]
bpword7=["fund","expert"]
bdword1=["transport","traffic","signal","transit",'route','bu','ticket']
bdword2=["monitor","project","consult"]
bdword3=["project","manag","consult"]

opword1=["implement", "park"]
opword2=["instal", "park"]
opword3=["manag", "bu","stand"]
opword4=["drive", "licens"]
rtword1=["consult", "bu"]
rtword2=['electr', 'buse' ]
rtword3=['electr', 'bu'] 
rtword4=['electr','vehicl']
stopword=['author','engin']
stopword2=['independ','engin']
stopword3=['authority', 'engin']
# stopword4=['supervis', 'consult', 'servic','lane']
stopword5=['consult','ae']
stopword6=['consult','ie']
stopword7=['dpr','lidar']
stopword8=['water','drainag']
stopword9=['system','integr','water']
stopword10=['water','expert']
stopword11=['water','suppli']
stopword12=['system','integr','child']
# stopword13=['system','integr','document']
stopword14=['satellit','track']
stopword15=['system','integr','secur']
stopword16=['shop','plan']
stopword17=['librari','academ','disabl','child','watershed','coal','drug','oil','fire','azur','insur','water','univers','lake','canal','geodetic']

for index, row in data1.iterrows():
    m=row["text_stem"]
    if all(word in m  for word in opword1):
        row["BU"]="OP" 
    if all(word in m  for word in opword2):
        row["BU"]="OP"
    if all(word in m  for word in opword3):
        row["BU"]="OP" 
    if all(word in m  for word in opword4):
        row["BU"]="OP" 
    if any(word in m  for word in bdword1):
        row["BU"]="BD2" 
    if all(word in m  for word in bdword2):
        row["BU"]="BD2" 
    if all(word in m  for word in bdword3):
        row["BU"]="BD2" 
    if all(word in m  for word in bdword4):
        row["BU"]="BD"  
    if all(word in m  for word in tpword3):
        row["BU"]="TP" 
    if all(word in m  for word in tpword5):
        row["BU"]="TP"
    if all(word in m  for word in tpword6):
        row["BU"]="TP" 
    if all(word in m  for word in tpword7):
        row["BU"]="TP"
    if all(word in m  for word in tpword8):
        row["BU"]="TP"
    if all(word in m  for word in tpword9):
        row["BU"]="TP" 
    if all(word in m  for word in tpword10):
        row["BU"]="TP"
    if all(word in m  for word in tpword11):
        row["BU"]="TP" 
    if all(word in m  for word in tpword12):
        row["BU"]="TP" 
    if all(word in m  for word in tpword13):
        row["BU"]="TP" 
    if all(word in m  for word in tpword14):
        row["BU"]="TP"
    if all(word in m  for word in tpword15):
        row["BU"]="TP" 
    if all(word in m  for word in tpword16):
        row["BU"]="TP"   
    if all(word in m  for word in tpword18):
        row["BU"]="TP" 
    if all(word in m  for word in tpword19):
        row["BU"]="TP" 
    if all(word in m  for word in tpword20):
        row["BU"]="TP"
    if all(word in m  for word in tpword21):
        row["BU"]="TP"  
    if all(word in m  for word in tpword22):
        row["BU"]="TP" 
    if all(word in m  for word in tpword23):
        row["BU"]="TP" 
    if all(word in m  for word in tpword24):
        row["BU"]="TP"  
    if all(word in m  for word in tpword25):
        row["BU"]="TP" 
    if all(word in m  for word in tpword26):
        row["BU"]="TP" 
    if all(word in m  for word in tpword27):
        row["BU"]="TP"
    if all(word in m  for word in tpword28):
        row["BU"]="TP"     
    if all(word in m  for word in rtword1):
        row["BU"]="RT" 
    if all(word in m  for word in rtword2):
        row["BU"]="RT" 
    if all(word in m  for word in rtword3):
        row["BU"]="RT" 
    if all(word in m  for word in rtword4):
        row["BU"]="RT"   
    if all(word in m  for word in ttsword5 ):
        row["BU"]="TTS" 

    if all(word in m  for word in ttsword7 ):
        row["BU"]="TTS"         
#     if all(word in m  for word in ttsword8 ):
#         row["BU"]="TTS"      
    if all(word in m  for word in enggword ):
        row["BU"]="Engg"
    if all(word in m  for word in enggword1 ):
        row["BU"]="Engg"  
    if all(word in m  for word in enggword2 ):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword3 ):
        row["BU"]="Engg"     
    if all(word in m  for word in enggword4 ):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword5 ):
        row["BU"]="Engg"  
    if all(word in m  for word in enggword6 ):
        row["BU"]="Engg"
    if all(word in m  for word in enggword7 ):
        row["BU"]="Engg"
    if all(word in m  for word in enggword8 ):
        row["BU"]="Engg"
    if all(word in m  for word in enggword9 ):
        row["BU"]="Engg"  
    if all(word in m  for word in enggword10 ):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword11 ):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword12 ):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword13):
        row["BU"]="Engg"     
    if all(word in m  for word in enggword14):
        row["BU"]="Engg"
    if all(word in m  for word in enggword15):
        row["BU"]="Engg"
    if all(word in m  for word in enggword16):
        row["BU"]="Engg"   
    if all(word in m  for word in enggword17):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword18):
        row["BU"]="Engg"  
    if all(word in m  for word in enggword19):
        row["BU"]="Engg"  
    if all(word in m  for word in enggword20):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword21):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword22):
        row["BU"]="Engg"
    if all(word in m  for word in enggword23):
        row["BU"]="Engg"
#     if all(word in m  for word in enggword24):
#         row["BU"]="Engg"
    if all(word in m  for word in enggword25):
        row["BU"]="Engg"
    if all(word in m  for word in enggword26):
        row["BU"]="Engg"
    if all(word in m  for word in enggword27):
        row["BU"]="Engg"  
    if all(word in m  for word in enggword28):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword29):
        row["BU"]="Engg"   
    if all(word in m  for word in enggword30):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword31):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword32):
        row["BU"]="Engg"
    if all(word in m  for word in enggword33):
        row["BU"]="Engg"
    if all(word in m  for word in enggword34):
        row["BU"]="Engg"
    if all(word in m  for word in enggword35):
        row["BU"]="Engg"  
    if all(word in m  for word in enggword36):
        row["BU"]="Engg"
    if all(word in m  for word in enggword37):
        row["BU"]="Engg"
    if all(word in m  for word in enggword38):
        row["BU"]="Engg"
    if all(word in m  for word in enggword39):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword40):
        row["BU"]="Engg"
    if all(word in m  for word in enggword41):
        row["BU"]="Engg"  
    if all(word in m  for word in enggword42):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword43):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword44):
        row["BU"]="Engg"  
    if all(word in m  for word in enggword45):
        row["BU"]="Engg"
    if all(word in m  for word in enggword46):
        row["BU"]="Engg"
    if all(word in m  for word in enggword47):
        row["BU"]="Engg"
    if all(word in m  for word in enggword48):
        row["BU"]="Engg" 
    if all(word in m  for word in enggword49):
        row["BU"]="Engg"     
    if all(word in m  for word in tpword2):
        row["BU"]="TP"  
    if all(word in m  for word in tpword19):
        row["BU"]="TP"     
    if all(word in m  for word in railword ):
        row["BU"]="Rail" 
    if all(word in m  for word in railword2 ):
        row["BU"]="Rail"  
    if all(word in m  for word in railword3 ):
        row["BU"]="Rail"    
    if all(word in m  for word in railword4 ):
        row["BU"]="Rail" 
    if all(word in m  for word in railword5 ):
        row["BU"]="Rail"  
    if all(word in m  for word in railword6 ):
        row["BU"]="Rail"  
    if all(word in m  for word in railword7 ):
        row["BU"]="Rail" 
    if all(word in m  for word in railword8 ):
        row["BU"]="Rail"
    if all(word in m  for word in railword9 ):
        row["BU"]="Rail" 
    if all(word in m  for word in railword10 ):
        row["BU"]="Rail"  
    if all(word in m  for word in railword11 ):
        row["BU"]="Rail" 
    if all(word in m  for word in railword12 ):
        row["BU"]="Rail"  
    if all(word in m  for word in railword13 ):
        row["BU"]="Rail"
    if all(word in m  for word in railword14):
        row["BU"]="Rail" 
    if all(word in m  for word in railword15):
        row["BU"]="Rail"     
    if all(word in m  for word in tpword4):
        row["BU"]="TP"     
#     if all(word in m  for word in ttsword1 ):
#         row["BU"]="TTS" 

#     if all(word in m  for word in ttsword2 ):
#         row["BU"]="TTS"     
    if all(word in m  for word in itsword1 ):
        row["BU"]="ITS"
    if all(word in m  for word in itsword2 ):
        row["BU"]="ITS" 
    if all(word in m  for word in itsword3 ):
        row["BU"]="ITS"
    if all(word in m  for word in itsword4 ):
        row["BU"]="ITS" 
    if all(word in m  for word in itsword5 ):
        row["BU"]="ITS" 
    if all(word in m  for word in itsword6 ):
        row["BU"]="ITS"
    if all(word in m  for word in itsword8 ):
        row["BU"]="ITS"
    if all(word in m  for word in itsword9 ):
        row["BU"]="ITS" 
    if all(word in m  for word in itsword10 ):
        row["BU"]="ITS"    
    if all(word in m  for word in itword10 ):
        row["BU"]="IT"   
    if all(word in m  for word in itword11 ):
        row["BU"]="IT"     
    if all(word in m  for word in itword1):
        row["BU"]="IT" 
    if all(word in m  for word in itword2):
        row["BU"]="IT"
    if all(word in m  for word in itword3):
        row["BU"]="IT" 
    if all(word in m  for word in ttsword4):
        row["BU"]="TTS"   
#     if all(word in m  for word in itword5):
#         row["BU"]="IT"
    if all(word in m  for word in itword6):
        row["BU"]="IT"   
    if all(word in m  for word in ttsword7):
        row["BU"]="TTS"     
    if all(word in m  for word in itsword7 ):
        row["BU"]="ITS"     
    if all(word in m  for word in tpword1):
        row["BU"]="TP"    
    if all(word in m  for word in erpword1):
        row["BU"]="TTS" 
    if all(word in m  for word in bpword2):
        row["BU"]="BP" 
    if all(word in m  for word in bpword4):
        row["BU"]="BP"
    if all(word in m  for word in bpword5):
        row["BU"]="BP" 
    if all(word in m  for word in bpword6):
        row["BU"]="BP"  
    if all(word in m  for word in bpword7):
        row["BU"]="BP"    

    if all(word in m  for word in ttsword6 ):
        row["BU"]="TTS"    
#     if all(word in m  for word in stopword):
#         row["BU"]=" " 
#     if all(word in m  for word in stopword2):
#         row["BU"]=" " 
#     if all(word in m  for word in stopword3):
#         row["BU"]=" "  
#     if all(word in m  for word in stopword4):
#         row["BU"]=" " 
#     if all(word in m  for word in stopword5):
#         row["BU"]=" " 
#     if all(word in m  for word in stopword6):
#         row["BU"]=" " 
#     if all(word in m  for word in stopword7):
#         row["BU"]=" " 
    if all(word in m  for word in stopword8):
        row["BU"]=" " 
    if all(word in m  for word in stopword9):
        row["BU"]=" "
    if all(word in m  for word in stopword10):
        row["BU"]=" "
    if all(word in m  for word in stopword11):
        row["BU"]=" " 
#     if all(word in m  for word in stopword12):
#         row["BU"]=" "
#     if all(word in m  for word in stopword13):
#         row["BU"]=" " 
#     if all(word in m  for word in stopword14):
#         row["BU"]=" " 
#     if all(word in m  for word in stopword15):
#         row["BU"]=" "  
#     if all(word in m  for word in stopword16):
#         row["BU"]=" "
    if any(word in m  for word in stopword17):
        row["BU"]=" " 
    if all(word in m  for word in bpword3):
        row["BU"]="BP" 
    if all(word in m  for word in bpword1):
        row["BU"]="BP"  
    if all(word in m  for word in tpword9):
        row["BU"]="TP"     
# data1[["Project Title","Project Location","BU","Description","Estimated Cost","Organisation","text_stem"]]

In [13]:
data1.head(50)

Title,Bidding Type,Project Title,Description,Organisation,Project Location,Client Address,Related Documents :,Deadline,Estimated Cost,Date,DescriptionA,text_stem,BU
0,Domestic Competitive Bidding,1.India - Rate Contract For Digital Signature Certificate Class Iii Only Sign Single Token With 2 Years Valid,"Tenders are invited for Rate contract for digital signature certificate class iii only sign single token with 2 years valid,Type: Open,Eligibility Criteria : -",EAST COAST RAILWAY,India,"Nandan Kanan Rd, Chandrasekharpur, Bhubaneshwar - 751017 , Orissa",Original Document : Document,2018-08-06 00:00:00,None,"Thursday, Jul 12, 2018",tenders are invited for rate contract for digital signature certificate class iii only sign single token with 2 years valid type open eligibility criteria,"[tender, are, invit, for, rate, contract, for, digit, signatur, certif, class, iii, onli, sign, singl, token, with, 2, year, valid, type, open, elig, criteria]",
1,Domestic Competitive Bidding,"2.India - Preparation Of Gis Based Master Plan For Karaikal, Mane & Yanam Planning Areas Of U.t. Of Puducherry","Tenders are invited for Preparation of GIS based Master Plan for Karaikal, Mane & Yanam Planning Areas of U.T. of Puducherry",TOWN AND COUNTRY PLANNING DEPARTMENT,India,"Jawahar Nagar Main Road, Jawahar Nagar, Boomianpet, Puducherry, 605005,Tel: +91-0413-2201952,Email_id: ctptcp.pon@nic.in",Original Document,2018-08-03 00:00:00,None,"Thursday, Jul 12, 2018",tenders are invited for preparation of gis based master plan for karaikal mane yanam planning areas of u t of puducherry,"[tender, are, invit, for, prepar, of, gi, base, master, plan, for, karaik, mane, yanam, plan, area, of, u, t, of, puducherri]",TP
2,Domestic Competitive Bidding,3.India - P/e New Street Light Point Of L.e.d From Old Octroi Post To City At Sarainaga Raod.,"Tenders are invited for P/e new street light point of l.e.d from old octroi post to city at sarainaga raod.,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No Eligibility Criteria : Please refer Tender documents. Document cost : INR 100 EMD value : INR 6000 Document Purcha...",DEPARTMENT OF LOCAL GOVERNMENT,India,"Director - Local Government||Regional Deputy Director - Bathinda||Nagar Panchayat - Bariwala, Np Bariwala",Original Document : Bid Document,2018-07-25 00:00:00,"INR, 300000","Thursday, Jul 12, 2018",tenders are invited for p e new street light point of l e d from old octroi post to city at sarainaga raod tender type open tender form of contract percentage tender category works no of covers 2 general technical evaluation allowed no itemwise technical evaluation allowed no payment mode online is multi currency allowed for boq no is multi currency allowed for fee no eligibility criteria please refer tender documents document cost inr 100 emd value inr 6000 document purcha...,"[tender, are, invit, for, p, e, new, street, light, point, of, l, e, d, from, old, octroi, post, to, citi, at, sarainaga, raod, tender, type, open, tender, form, of, contract, percentag, tender, categori, work, no, of, cover, 2, gener, technic, evalu, allow, no, itemwis, technic, evalu, allow, no, payment, mode, onlin, is, multi, currenc, allow, for, boq, no, is, multi, currenc, allow, for, fee, no, elig, criteria, pleas, refer, tender, document, document, cost, inr, 100, emd, valu, inr, 600...",
3,Domestic Competitive Bidding,4.India - Supply Of Goods For Street Light Maintance,"Tenders are invited for Supply of goods for street light maintance,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No Eligibilit

In [14]:
# from nltk import PorterStemmer
PorterStemmer().stem('university')

u'univers'

In [15]:
data1["BD"]=" "
data1.head()

Title,Bidding Type,Project Title,Description,Organisation,Project Location,Client Address,Related Documents :,Deadline,Estimated Cost,Date,DescriptionA,text_stem,BU,BD
0,Domestic Competitive Bidding,1.India - Rate Contract For Digital Signature Certificate Class Iii Only Sign Single Token With 2 Years Valid,"Tenders are invited for Rate contract for digital signature certificate class iii only sign single token with 2 years valid,Type: Open,Eligibility Criteria : -",EAST COAST RAILWAY,India,"Nandan Kanan Rd, Chandrasekharpur, Bhubaneshwar - 751017 , Orissa",Original Document : Document,2018-08-06 00:00:00,None,"Thursday, Jul 12, 2018",tenders are invited for rate contract for digital signature certificate class iii only sign single token with 2 years valid type open eligibility criteria,"[tender, are, invit, for, rate, contract, for, digit, signatur, certif, class, iii, onli, sign, singl, token, with, 2, year, valid, type, open, elig, criteria]",,
1,Domestic Competitive Bidding,"2.India - Preparation Of Gis Based Master Plan For Karaikal, Mane & Yanam Planning Areas Of U.t. Of Puducherry","Tenders are invited for Preparation of GIS based Master Plan for Karaikal, Mane & Yanam Planning Areas of U.T. of Puducherry",TOWN AND COUNTRY PLANNING DEPARTMENT,India,"Jawahar Nagar Main Road, Jawahar Nagar, Boomianpet, Puducherry, 605005,Tel: +91-0413-2201952,Email_id: ctptcp.pon@nic.in",Original Document,2018-08-03 00:00:00,None,"Thursday, Jul 12, 2018",tenders are invited for preparation of gis based master plan for karaikal mane yanam planning areas of u t of puducherry,"[tender, are, invit, for, prepar, of, gi, base, master, plan, for, karaik, mane, yanam, plan, area, of, u, t, of, puducherri]",TP,
2,Domestic Competitive Bidding,3.India - P/e New Street Light Point Of L.e.d From Old Octroi Post To City At Sarainaga Raod.,"Tenders are invited for P/e new street light point of l.e.d from old octroi post to city at sarainaga raod.,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No Eligibility Criteria : Please refer Tender documents. Document cost : INR 100 EMD value : INR 6000 Document Purcha...",DEPARTMENT OF LOCAL GOVERNMENT,India,"Director - Local Government||Regional Deputy Director - Bathinda||Nagar Panchayat - Bariwala, Np Bariwala",Original Document : Bid Document,2018-07-25 00:00:00,"INR, 300000","Thursday, Jul 12, 2018",tenders are invited for p e new street light point of l e d from old octroi post to city at sarainaga raod tender type open tender form of contract percentage tender category works no of covers 2 general technical evaluation allowed no itemwise technical evaluation allowed no payment mode online is multi currency allowed for boq no is multi currency allowed for fee no eligibility criteria please refer tender documents document cost inr 100 emd value inr 6000 document purcha...,"[tender, are, invit, for, p, e, new, street, light, point, of, l, e, d, from, old, octroi, post, to, citi, at, sarainaga, raod, tender, type, open, tender, form, of, contract, percentag, tender, categori, work, no, of, cover, 2, gener, technic, evalu, allow, no, itemwis, technic, evalu, allow, no, payment, mode, onlin, is, multi, currenc, allow, for, boq, no, is, multi, currenc, allow, for, fee, no, elig, criteria, pleas, refer, tender, document, document, cost, inr, 100, emd, valu, inr, 600...",,
3,Domestic Competitive Bidding,4.India - Supply Of Goods For Street Light Maintance,"Tenders are invited for Supply of goods for street light maintance,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No Elig

In [16]:
bu1=['TP','OP']
bu2=['Rail','Engg','BP']
bu3=['IT','TTS']
bu4=['IT','ITS']
bu5=['RT']
bu6=['BD']
bu7=['BD2']
for index, row in data1.iterrows():
    m=row["BU"]
    if m in bu1:
        row['BD']='Ashish'   
    if m in bu2:
        row['BD']='Anand'      
    if m in bu3:
        row['BD']='Saumya' 
    if m in bu4:
        row['BD']='Gurpreet'  
    if m in bu5:
        row['BD']='Sudeep'  
    if m in bu6:
        row['BD']='All'
    if m in bu7:
        row['BD']='Check'     

In [17]:
data1['Estimated Cost'] = data1['Estimated Cost'].str.replace("INR,", 'Rs.')
data1.head()

Title,Bidding Type,Project Title,Description,Organisation,Project Location,Client Address,Related Documents :,Deadline,Estimated Cost,Date,DescriptionA,text_stem,BU,BD
0,Domestic Competitive Bidding,1.India - Rate Contract For Digital Signature Certificate Class Iii Only Sign Single Token With 2 Years Valid,"Tenders are invited for Rate contract for digital signature certificate class iii only sign single token with 2 years valid,Type: Open,Eligibility Criteria : -",EAST COAST RAILWAY,India,"Nandan Kanan Rd, Chandrasekharpur, Bhubaneshwar - 751017 , Orissa",Original Document : Document,2018-08-06 00:00:00,None,"Thursday, Jul 12, 2018",tenders are invited for rate contract for digital signature certificate class iii only sign single token with 2 years valid type open eligibility criteria,"[tender, are, invit, for, rate, contract, for, digit, signatur, certif, class, iii, onli, sign, singl, token, with, 2, year, valid, type, open, elig, criteria]",,
1,Domestic Competitive Bidding,"2.India - Preparation Of Gis Based Master Plan For Karaikal, Mane & Yanam Planning Areas Of U.t. Of Puducherry","Tenders are invited for Preparation of GIS based Master Plan for Karaikal, Mane & Yanam Planning Areas of U.T. of Puducherry",TOWN AND COUNTRY PLANNING DEPARTMENT,India,"Jawahar Nagar Main Road, Jawahar Nagar, Boomianpet, Puducherry, 605005,Tel: +91-0413-2201952,Email_id: ctptcp.pon@nic.in",Original Document,2018-08-03 00:00:00,None,"Thursday, Jul 12, 2018",tenders are invited for preparation of gis based master plan for karaikal mane yanam planning areas of u t of puducherry,"[tender, are, invit, for, prepar, of, gi, base, master, plan, for, karaik, mane, yanam, plan, area, of, u, t, of, puducherri]",TP,Ashish
2,Domestic Competitive Bidding,3.India - P/e New Street Light Point Of L.e.d From Old Octroi Post To City At Sarainaga Raod.,"Tenders are invited for P/e new street light point of l.e.d from old octroi post to city at sarainaga raod.,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No Eligibility Criteria : Please refer Tender documents. Document cost : INR 100 EMD value : INR 6000 Document Purcha...",DEPARTMENT OF LOCAL GOVERNMENT,India,"Director - Local Government||Regional Deputy Director - Bathinda||Nagar Panchayat - Bariwala, Np Bariwala",Original Document : Bid Document,2018-07-25 00:00:00,Rs. 300000,"Thursday, Jul 12, 2018",tenders are invited for p e new street light point of l e d from old octroi post to city at sarainaga raod tender type open tender form of contract percentage tender category works no of covers 2 general technical evaluation allowed no itemwise technical evaluation allowed no payment mode online is multi currency allowed for boq no is multi currency allowed for fee no eligibility criteria please refer tender documents document cost inr 100 emd value inr 6000 document purcha...,"[tender, are, invit, for, p, e, new, street, light, point, of, l, e, d, from, old, octroi, post, to, citi, at, sarainaga, raod, tender, type, open, tender, form, of, contract, percentag, tender, categori, work, no, of, cover, 2, gener, technic, evalu, allow, no, itemwis, technic, evalu, allow, no, payment, mode, onlin, is, multi, currenc, allow, for, boq, no, is, multi, currenc, allow, for, fee, no, elig, criteria, pleas, refer, tender, document, document, cost, inr, 100, emd, valu, inr, 600...",,
3,Domestic Competitive Bidding,4.India - Supply Of Goods For Street Light Maintance,"Tenders are invited for Supply of goods for street light maintance,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No E

In [18]:
data1.dtypes

Title
Bidding Type           object
Project Title          object
Description            object
Organisation           object
Project Location       object
Client Address         object
Related Documents :    object
Deadline               object
Estimated Cost         object
Date                   object
DescriptionA           object
text_stem              object
BU                     object
BD                     object
dtype: object

In [19]:
# data1['Deadline'] = pd.to_datetime(data1['Deadline'],format='%d%m%Y' )
data1['Deadline'] = pd.to_datetime(data1['Deadline'])
data1.head()

Title,Bidding Type,Project Title,Description,Organisation,Project Location,Client Address,Related Documents :,Deadline,Estimated Cost,Date,DescriptionA,text_stem,BU,BD
0,Domestic Competitive Bidding,1.India - Rate Contract For Digital Signature Certificate Class Iii Only Sign Single Token With 2 Years Valid,"Tenders are invited for Rate contract for digital signature certificate class iii only sign single token with 2 years valid,Type: Open,Eligibility Criteria : -",EAST COAST RAILWAY,India,"Nandan Kanan Rd, Chandrasekharpur, Bhubaneshwar - 751017 , Orissa",Original Document : Document,2018-08-06,None,"Thursday, Jul 12, 2018",tenders are invited for rate contract for digital signature certificate class iii only sign single token with 2 years valid type open eligibility criteria,"[tender, are, invit, for, rate, contract, for, digit, signatur, certif, class, iii, onli, sign, singl, token, with, 2, year, valid, type, open, elig, criteria]",,
1,Domestic Competitive Bidding,"2.India - Preparation Of Gis Based Master Plan For Karaikal, Mane & Yanam Planning Areas Of U.t. Of Puducherry","Tenders are invited for Preparation of GIS based Master Plan for Karaikal, Mane & Yanam Planning Areas of U.T. of Puducherry",TOWN AND COUNTRY PLANNING DEPARTMENT,India,"Jawahar Nagar Main Road, Jawahar Nagar, Boomianpet, Puducherry, 605005,Tel: +91-0413-2201952,Email_id: ctptcp.pon@nic.in",Original Document,2018-08-03,None,"Thursday, Jul 12, 2018",tenders are invited for preparation of gis based master plan for karaikal mane yanam planning areas of u t of puducherry,"[tender, are, invit, for, prepar, of, gi, base, master, plan, for, karaik, mane, yanam, plan, area, of, u, t, of, puducherri]",TP,Ashish
2,Domestic Competitive Bidding,3.India - P/e New Street Light Point Of L.e.d From Old Octroi Post To City At Sarainaga Raod.,"Tenders are invited for P/e new street light point of l.e.d from old octroi post to city at sarainaga raod.,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No Eligibility Criteria : Please refer Tender documents. Document cost : INR 100 EMD value : INR 6000 Document Purcha...",DEPARTMENT OF LOCAL GOVERNMENT,India,"Director - Local Government||Regional Deputy Director - Bathinda||Nagar Panchayat - Bariwala, Np Bariwala",Original Document : Bid Document,2018-07-25,Rs. 300000,"Thursday, Jul 12, 2018",tenders are invited for p e new street light point of l e d from old octroi post to city at sarainaga raod tender type open tender form of contract percentage tender category works no of covers 2 general technical evaluation allowed no itemwise technical evaluation allowed no payment mode online is multi currency allowed for boq no is multi currency allowed for fee no eligibility criteria please refer tender documents document cost inr 100 emd value inr 6000 document purcha...,"[tender, are, invit, for, p, e, new, street, light, point, of, l, e, d, from, old, octroi, post, to, citi, at, sarainaga, raod, tender, type, open, tender, form, of, contract, percentag, tender, categori, work, no, of, cover, 2, gener, technic, evalu, allow, no, itemwis, technic, evalu, allow, no, payment, mode, onlin, is, multi, currenc, allow, for, boq, no, is, multi, currenc, allow, for, fee, no, elig, criteria, pleas, refer, tender, document, document, cost, inr, 100, emd, valu, inr, 600...",,
3,Domestic Competitive Bidding,4.India - Supply Of Goods For Street Light Maintance,"Tenders are invited for Supply of goods for street light maintance,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No Eligibility Criteria : As pe

In [20]:
# data1["TenderScrutinyDate"]=pd.to_datetime("2018-06-22" )
data1["Date"]=pd.to_datetime(data1.Date)
data1.head()

Title,Bidding Type,Project Title,Description,Organisation,Project Location,Client Address,Related Documents :,Deadline,Estimated Cost,Date,DescriptionA,text_stem,BU,BD
0,Domestic Competitive Bidding,1.India - Rate Contract For Digital Signature Certificate Class Iii Only Sign Single Token With 2 Years Valid,"Tenders are invited for Rate contract for digital signature certificate class iii only sign single token with 2 years valid,Type: Open,Eligibility Criteria : -",EAST COAST RAILWAY,India,"Nandan Kanan Rd, Chandrasekharpur, Bhubaneshwar - 751017 , Orissa",Original Document : Document,2018-08-06,None,2018-07-12,tenders are invited for rate contract for digital signature certificate class iii only sign single token with 2 years valid type open eligibility criteria,"[tender, are, invit, for, rate, contract, for, digit, signatur, certif, class, iii, onli, sign, singl, token, with, 2, year, valid, type, open, elig, criteria]",,
1,Domestic Competitive Bidding,"2.India - Preparation Of Gis Based Master Plan For Karaikal, Mane & Yanam Planning Areas Of U.t. Of Puducherry","Tenders are invited for Preparation of GIS based Master Plan for Karaikal, Mane & Yanam Planning Areas of U.T. of Puducherry",TOWN AND COUNTRY PLANNING DEPARTMENT,India,"Jawahar Nagar Main Road, Jawahar Nagar, Boomianpet, Puducherry, 605005,Tel: +91-0413-2201952,Email_id: ctptcp.pon@nic.in",Original Document,2018-08-03,None,2018-07-12,tenders are invited for preparation of gis based master plan for karaikal mane yanam planning areas of u t of puducherry,"[tender, are, invit, for, prepar, of, gi, base, master, plan, for, karaik, mane, yanam, plan, area, of, u, t, of, puducherri]",TP,Ashish
2,Domestic Competitive Bidding,3.India - P/e New Street Light Point Of L.e.d From Old Octroi Post To City At Sarainaga Raod.,"Tenders are invited for P/e new street light point of l.e.d from old octroi post to city at sarainaga raod.,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No Eligibility Criteria : Please refer Tender documents. Document cost : INR 100 EMD value : INR 6000 Document Purcha...",DEPARTMENT OF LOCAL GOVERNMENT,India,"Director - Local Government||Regional Deputy Director - Bathinda||Nagar Panchayat - Bariwala, Np Bariwala",Original Document : Bid Document,2018-07-25,Rs. 300000,2018-07-12,tenders are invited for p e new street light point of l e d from old octroi post to city at sarainaga raod tender type open tender form of contract percentage tender category works no of covers 2 general technical evaluation allowed no itemwise technical evaluation allowed no payment mode online is multi currency allowed for boq no is multi currency allowed for fee no eligibility criteria please refer tender documents document cost inr 100 emd value inr 6000 document purcha...,"[tender, are, invit, for, p, e, new, street, light, point, of, l, e, d, from, old, octroi, post, to, citi, at, sarainaga, raod, tender, type, open, tender, form, of, contract, percentag, tender, categori, work, no, of, cover, 2, gener, technic, evalu, allow, no, itemwis, technic, evalu, allow, no, payment, mode, onlin, is, multi, currenc, allow, for, boq, no, is, multi, currenc, allow, for, fee, no, elig, criteria, pleas, refer, tender, document, document, cost, inr, 100, emd, valu, inr, 600...",,
3,Domestic Competitive Bidding,4.India - Supply Of Goods For Street Light Maintance,"Tenders are invited for Supply of goods for street light maintance,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No Eligibility Criteria : As per DNIT EMD value : INR 3000 Document Purch

In [21]:
data1["DaysLeft"] = data1['Deadline'].sub(data1["Date"], axis=0)
data1.head()

Title,Bidding Type,Project Title,Description,Organisation,Project Location,Client Address,Related Documents :,Deadline,Estimated Cost,Date,DescriptionA,text_stem,BU,BD,DaysLeft
0,Domestic Competitive Bidding,1.India - Rate Contract For Digital Signature Certificate Class Iii Only Sign Single Token With 2 Years Valid,"Tenders are invited for Rate contract for digital signature certificate class iii only sign single token with 2 years valid,Type: Open,Eligibility Criteria : -",EAST COAST RAILWAY,India,"Nandan Kanan Rd, Chandrasekharpur, Bhubaneshwar - 751017 , Orissa",Original Document : Document,2018-08-06,None,2018-07-12,tenders are invited for rate contract for digital signature certificate class iii only sign single token with 2 years valid type open eligibility criteria,"[tender, are, invit, for, rate, contract, for, digit, signatur, certif, class, iii, onli, sign, singl, token, with, 2, year, valid, type, open, elig, criteria]",,,25 days
1,Domestic Competitive Bidding,"2.India - Preparation Of Gis Based Master Plan For Karaikal, Mane & Yanam Planning Areas Of U.t. Of Puducherry","Tenders are invited for Preparation of GIS based Master Plan for Karaikal, Mane & Yanam Planning Areas of U.T. of Puducherry",TOWN AND COUNTRY PLANNING DEPARTMENT,India,"Jawahar Nagar Main Road, Jawahar Nagar, Boomianpet, Puducherry, 605005,Tel: +91-0413-2201952,Email_id: ctptcp.pon@nic.in",Original Document,2018-08-03,None,2018-07-12,tenders are invited for preparation of gis based master plan for karaikal mane yanam planning areas of u t of puducherry,"[tender, are, invit, for, prepar, of, gi, base, master, plan, for, karaik, mane, yanam, plan, area, of, u, t, of, puducherri]",TP,Ashish,22 days
2,Domestic Competitive Bidding,3.India - P/e New Street Light Point Of L.e.d From Old Octroi Post To City At Sarainaga Raod.,"Tenders are invited for P/e new street light point of l.e.d from old octroi post to city at sarainaga raod.,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No Eligibility Criteria : Please refer Tender documents. Document cost : INR 100 EMD value : INR 6000 Document Purcha...",DEPARTMENT OF LOCAL GOVERNMENT,India,"Director - Local Government||Regional Deputy Director - Bathinda||Nagar Panchayat - Bariwala, Np Bariwala",Original Document : Bid Document,2018-07-25,Rs. 300000,2018-07-12,tenders are invited for p e new street light point of l e d from old octroi post to city at sarainaga raod tender type open tender form of contract percentage tender category works no of covers 2 general technical evaluation allowed no itemwise technical evaluation allowed no payment mode online is multi currency allowed for boq no is multi currency allowed for fee no eligibility criteria please refer tender documents document cost inr 100 emd value inr 6000 document purcha...,"[tender, are, invit, for, p, e, new, street, light, point, of, l, e, d, from, old, octroi, post, to, citi, at, sarainaga, raod, tender, type, open, tender, form, of, contract, percentag, tender, categori, work, no, of, cover, 2, gener, technic, evalu, allow, no, itemwis, technic, evalu, allow, no, payment, mode, onlin, is, multi, currenc, allow, for, boq, no, is, multi, currenc, allow, for, fee, no, elig, criteria, pleas, refer, tender, document, document, cost, inr, 100, emd, valu, inr, 600...",,,13 days
3,Domestic Competitive Bidding,4.India - Supply Of Goods For Street Light Maintance,"Tenders are invited for Supply of goods for street light maintance,Tender Type: Open tender,Form Of Contract: Percentage,Tender Category: Works,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Online,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No Eligibility Criteria : As per DNIT EM

In [22]:
columnsTitles=["Description",'Bidding Type',"Project Title","Organisation","Project Location","Client Address","Estimated Cost","BU","BD","Deadline","Date","DaysLeft",'Related Documents :']
data1=data1.reindex(columns=columnsTitles)

In [23]:
data1[data1.BU != " "]

Title,Description,Bidding Type,Project Title,Organisation,Project Location,Client Address,Estimated Cost,BU,BD,Deadline,Date,DaysLeft,Related Documents :
1,"Tenders are invited for Preparation of GIS based Master Plan for Karaikal, Mane & Yanam Planning Areas of U.T. of Puducherry",Domestic Competitive Bidding,"2.India - Preparation Of Gis Based Master Plan For Karaikal, Mane & Yanam Planning Areas Of U.t. Of Puducherry",TOWN AND COUNTRY PLANNING DEPARTMENT,India,"Jawahar Nagar Main Road, Jawahar Nagar, Boomianpet, Puducherry, 605005,Tel: +91-0413-2201952,Email_id: ctptcp.pon@nic.in",None,TP,Ashish,2018-08-03,2018-07-12,22 days,Original Document
5,"Tenders are invited for Consultancy services for preparation of feasibility study and detailed project report in the state of punjab for improvement of existing road geometric design safety measures and other related works for the road nh154a from km 0.00 to 39.36,Product Category: Consultancy services,EMD Amount (INR): 2,00,000,Tender Type: Open tender,Bid Validity(Days): 120,Period Of Work(Days): 90,Location: As per tender documents,Document Download Start Date: 10-jul-2018 04:00 pm,Docume...",Domestic Competitive Bidding,6.India - Consultancy Services For Preparation Of Feasibility Study And Detailed Project Report In The State Of Punjab For Improvement Of Existing Road Geometric Design Safety Measures And Other Related Works F,MINISTRY OF ROAD TRANSPORT AND HIGHWAYS (MORTH),India,"P1 Delhi - Morth||Ro Chandigarh - Morth||Cwd-Ferozepur - Morth, O/O Of Xen Central Works Divin Pwd Bandr Pathankot",None,TP,Ashish,2018-07-31,2018-07-12,19 days,Original Document : Bid Document
16,"Tenders are invited for Supply and installation of intelligent transport system (its), for safe demonstration corridor (deeg - alwar - behror section of sh-14) under eprocurement system,Tender Type: Open tender,Form Of Contract: Item rate,Tender Category: Goods,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Offline,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No,Eligibility Criteria : Please refer ...",Domestic Competitive Bidding,"17.India - Supply And Installation Of Intelligent Transport System (its), For Safe Demonstration Corridor (",RAJASTHAN,India,"Se-Pmgsy-Rj, Chief Engineer(Pmgsy), P.W.D., Rajasthan, Jaipur",Rs. 46000000,ITS,Gurpreet,2018-07-30,2018-07-12,18 days,Original Document : Bid Document : Corrigendum 1
19,"Tenders are invited for Consultancy services for preparation of feasibility study ""c u m"" detailed project report study (fs DPR) for establishing bus port at salem city.,Product Category: Consultancy services,Tender Value: 30,00,000,EMD Amount (INR): 2,00,000,Tender Type: Open tender,Bid Validity(Days): 120,Period Of Work(Days): 90,Location: Salem,Document Download Start Date: 08-jun-2018 12:50 pm,Document Download End Date: 29-jun-2018 03:00 pm,Bid Submission Start Date: 08-jun-2018 12:55 p...",Domestic Competitive Bidding,"20.India - Consultancy Services For Preparation Of Feasibility Study ""c u m"" Detailed Project Report Study (fs Dpr) For Establishing Bus Port At Salem City.",MINISTRY OF ROAD TRANSPORT AND HIGHWAYS (MORTH),India,"P7 Delh - Morth||Ro Chennai - Morth||Nh Division-Salem - Morth, O/O The Superintending Engineer, National Highways Omalur Main Road Narasothipatty,Alagapuram (Po),Salem 636 004",Rs. 3000000,Engg,Anand,2018-07-18,2018-07-12,6 days,Original Document : Bid Document : Corrigendum 1 : Corrigendum 2 : Corrigendum 3
32,"Tenders are invited for Selection of consultant for preparation of comprehensive Mobility (traffic and transportation) plan for greater noida,Tender Type: Open tender,Form Of Contract: Lump-sum,Tender Category: Services,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Offline,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No Eligibility Criteria : REFER TENDER DOC

In [24]:
# from nltk import PorterStemmer
# PorterStemmer().stem('child')

In [25]:
data2 = data1[data1.BU != " "]
data2

Title,Description,Bidding Type,Project Title,Organisation,Project Location,Client Address,Estimated Cost,BU,BD,Deadline,Date,DaysLeft,Related Documents :
1,"Tenders are invited for Preparation of GIS based Master Plan for Karaikal, Mane & Yanam Planning Areas of U.T. of Puducherry",Domestic Competitive Bidding,"2.India - Preparation Of Gis Based Master Plan For Karaikal, Mane & Yanam Planning Areas Of U.t. Of Puducherry",TOWN AND COUNTRY PLANNING DEPARTMENT,India,"Jawahar Nagar Main Road, Jawahar Nagar, Boomianpet, Puducherry, 605005,Tel: +91-0413-2201952,Email_id: ctptcp.pon@nic.in",None,TP,Ashish,2018-08-03,2018-07-12,22 days,Original Document
5,"Tenders are invited for Consultancy services for preparation of feasibility study and detailed project report in the state of punjab for improvement of existing road geometric design safety measures and other related works for the road nh154a from km 0.00 to 39.36,Product Category: Consultancy services,EMD Amount (INR): 2,00,000,Tender Type: Open tender,Bid Validity(Days): 120,Period Of Work(Days): 90,Location: As per tender documents,Document Download Start Date: 10-jul-2018 04:00 pm,Docume...",Domestic Competitive Bidding,6.India - Consultancy Services For Preparation Of Feasibility Study And Detailed Project Report In The State Of Punjab For Improvement Of Existing Road Geometric Design Safety Measures And Other Related Works F,MINISTRY OF ROAD TRANSPORT AND HIGHWAYS (MORTH),India,"P1 Delhi - Morth||Ro Chandigarh - Morth||Cwd-Ferozepur - Morth, O/O Of Xen Central Works Divin Pwd Bandr Pathankot",None,TP,Ashish,2018-07-31,2018-07-12,19 days,Original Document : Bid Document
16,"Tenders are invited for Supply and installation of intelligent transport system (its), for safe demonstration corridor (deeg - alwar - behror section of sh-14) under eprocurement system,Tender Type: Open tender,Form Of Contract: Item rate,Tender Category: Goods,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Offline,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No,Eligibility Criteria : Please refer ...",Domestic Competitive Bidding,"17.India - Supply And Installation Of Intelligent Transport System (its), For Safe Demonstration Corridor (",RAJASTHAN,India,"Se-Pmgsy-Rj, Chief Engineer(Pmgsy), P.W.D., Rajasthan, Jaipur",Rs. 46000000,ITS,Gurpreet,2018-07-30,2018-07-12,18 days,Original Document : Bid Document : Corrigendum 1
19,"Tenders are invited for Consultancy services for preparation of feasibility study ""c u m"" detailed project report study (fs DPR) for establishing bus port at salem city.,Product Category: Consultancy services,Tender Value: 30,00,000,EMD Amount (INR): 2,00,000,Tender Type: Open tender,Bid Validity(Days): 120,Period Of Work(Days): 90,Location: Salem,Document Download Start Date: 08-jun-2018 12:50 pm,Document Download End Date: 29-jun-2018 03:00 pm,Bid Submission Start Date: 08-jun-2018 12:55 p...",Domestic Competitive Bidding,"20.India - Consultancy Services For Preparation Of Feasibility Study ""c u m"" Detailed Project Report Study (fs Dpr) For Establishing Bus Port At Salem City.",MINISTRY OF ROAD TRANSPORT AND HIGHWAYS (MORTH),India,"P7 Delh - Morth||Ro Chennai - Morth||Nh Division-Salem - Morth, O/O The Superintending Engineer, National Highways Omalur Main Road Narasothipatty,Alagapuram (Po),Salem 636 004",Rs. 3000000,Engg,Anand,2018-07-18,2018-07-12,6 days,Original Document : Bid Document : Corrigendum 1 : Corrigendum 2 : Corrigendum 3
32,"Tenders are invited for Selection of consultant for preparation of comprehensive Mobility (traffic and transportation) plan for greater noida,Tender Type: Open tender,Form Of Contract: Lump-sum,Tender Category: Services,No. of Covers: 2,General Technical Evaluation Allowed: No,ItemWise Technical Evaluation Allowed: No,Payment Mode: Offline,Is Multi Currency Allowed For BOQ: No,Is Multi Currency Allowed For Fee: No Eligibility Criteria : REFER TENDER DOC

In [26]:
data2.sort_values(["Project Location","BU"], inplace=True)

In [27]:
data2

Title,Description,Bidding Type,Project Title,Organisation,Project Location,Client Address,Estimated Cost,BU,BD,Deadline,Date,DaysLeft,Related Documents :
363,"Tenders are invited for provide background and information on Regional-national Civic Engagement Grants and to solicit,Description: 1) North West Region (Balkh, Faryab, Jawzjan, Sari pul, Samangan). (2) Southern Region (Kandahar, Helmand, Nimroz, Zabul, Uruzgan,• Develop the Capacity of provincial CSOs as per PY6 workplan, using coaching, mentoring, on the job training, and conducting training workshops. • Facilitate Networking and collaboration among CSOs and other stakeholders focusing on ...",Not Specified Domestic / International Bidding ?,3.Afghanistan - Provide Background And Information On Regional-national Civic Engagement Grants And To Solicit,COUNTERPART INTERNATIONAL – AFGHANISTAN,Afghanistan,"Kabul, Afghanistan,Tel: +93-21-700 154 933,Email_id: applications.acep@counterpart.org",None,TP,Ashish,2018-07-16,2018-07-12,4 days,Original Document
239,Tenders are invited for provision of traffic management services,Not Specified Domestic / International Bidding ?,3.Australia - Provision Of Traffic Management Services,CITY OF BUSSELTON,Australia,"Locked Bag 1, BUSSELTON WA 6280,Tel: +61-08) 9781 0444,Fax: +61-08) 9752 4958,Email_id: city@busselton.wa.gov.au",None,BD2,Check,2018-07-31,2018-07-12,19 days,Original Document
238,"Tenders are invited for Spray Seal Services, traffic Control and Associated Products,Tweed Shire Council (Principal) seeks to engage a suitably qualified and experienced Contractor to supply spray seal services, traffic control and associated products.,Whilst there is no minimum quantities of purchase of Services guaranteed under this Contract, The Principal estimates expenditure in the vicinity of $3.0M for the provision of sprayed bituminous surfacing services and associated products for t...",Not Specified Domestic / International Bidding ?,"2.Australia - Spray Seal Services, Traffic Control And Associated Products",TWEED SHIRE COUNCIL,Australia,"PO Box 816, Murwillumbah NSW 2484,Tel: +61-(02) 6670 2400,Email_id: tsc@tweed.nsw.gov.au",None,ITS,Gurpreet,2018-08-08,2018-07-12,27 days,Original Document
242,"Expressions of interest for Supervision of Project Works: ""Construction Adduction 1 (Ptap Jove Rancho - Tiquipaya - Cochabamba - Sacaba)"",Description The Services of the Consultant Will Have the Purpose of Supervising the Construction of Works of Adduction 1, Assuming All Those Functions and Operational Responsibilities for the Execution of the Service. The Planned Works Include: a Line of Adduction (24 Km, Dn 800mm in Hdpe and Ffd), Branches (7 Km, Dn 450 and 500mm), Five Tanks (Volumes Bet...",Not Specified Domestic / International Bidding ?,"3.Bolivia - Supervision Of Project Works: ""construction Adduction 1 (ptap Jove Rancho - Tiquipaya - Cochabamba - Sacaba)""",MINISTRY OF ENVIRONMENT AND WATER (MMAYA),Bolivia,"Zona Sur, Obrajes Av. 14 de septiembre corner street 8, Edif. MMAyA piso 6 La Paz, Murillo Bolivie,Tel: +591-2 2148233,Email_id: adquisiciones.ucp.paap@gmail.com",None,Engg,Anand,2018-08-07,2018-07-12,26 days,Original Document
248,"Tenders are invited for acquisition of hospital supplies (closed tracheal aspiration system, percutaneous introductory kit and others),Proclamation from: 12/07/2018 das 08:00 às 12:00 hs e das 13:00 às 16:00 hs,1 - closed system trawler systems closed traqual aspiration, ~y~ pulley culin, via irrigation with anti-reflex valve, graduated probe with colored brand / siliconized cover, valve suction valve with security lock, application endfranchal tube 12fr, sterility differential treatment: -a...",Not Specified Domestic / International Bidding ?,"9.Brazil - Acquisition Of Hospital Supplies (closed Tracheal Aspiration System, Percutaneous Introductory Kit And Others)",MINISTÉRIO DA SAÚDE,Brazil,"Av. Brasil, Nº 500, Sao Cristovao - - Rio de Janeiro (RJ)",None,BD2,Check,2018-07-25,2018-07-12,13 days,Origina

In [28]:
# nreg=[]
nreg=['Australia','Taiwan Province of China','Brazil','Canada','Germany','Denmark','France','United States','Italy','Finland','Czech Republic','China','Belgium','Belarus','New Zealand','Russian Federation','Japan']

In [29]:
# final=data2.loc[data2["Project Location"] != ['Australia','Canada','Germany','France','United States','Italy','Finland','Czech Republic','China','Belgium','Belarus']]
# final.drop(['text_stem','Project Title'], axis=1, inplace=True)
final=data2[~data2["Project Location"].isin(nreg)]

In [30]:
final = final.reset_index(drop=True)
final.head()

Title,Description,Bidding Type,Project Title,Organisation,Project Location,Client Address,Estimated Cost,BU,BD,Deadline,Date,DaysLeft,Related Documents :
0,"Tenders are invited for provide background and information on Regional-national Civic Engagement Grants and to solicit,Description: 1) North West Region (Balkh, Faryab, Jawzjan, Sari pul, Samangan). (2) Southern Region (Kandahar, Helmand, Nimroz, Zabul, Uruzgan,• Develop the Capacity of provincial CSOs as per PY6 workplan, using coaching, mentoring, on the job training, and conducting training workshops. • Facilitate Networking and collaboration among CSOs and other stakeholders focusing on ...",Not Specified Domestic / International Bidding ?,3.Afghanistan - Provide Background And Information On Regional-national Civic Engagement Grants And To Solicit,COUNTERPART INTERNATIONAL – AFGHANISTAN,Afghanistan,"Kabul, Afghanistan,Tel: +93-21-700 154 933,Email_id: applications.acep@counterpart.org",None,TP,Ashish,2018-07-16,2018-07-12,4 days,Original Document
1,"Expressions of interest for Supervision of Project Works: ""Construction Adduction 1 (Ptap Jove Rancho - Tiquipaya - Cochabamba - Sacaba)"",Description The Services of the Consultant Will Have the Purpose of Supervising the Construction of Works of Adduction 1, Assuming All Those Functions and Operational Responsibilities for the Execution of the Service. The Planned Works Include: a Line of Adduction (24 Km, Dn 800mm in Hdpe and Ffd), Branches (7 Km, Dn 450 and 500mm), Five Tanks (Volumes Bet...",Not Specified Domestic / International Bidding ?,"3.Bolivia - Supervision Of Project Works: ""construction Adduction 1 (ptap Jove Rancho - Tiquipaya - Cochabamba - Sacaba)""",MINISTRY OF ENVIRONMENT AND WATER (MMAYA),Bolivia,"Zona Sur, Obrajes Av. 14 de septiembre corner street 8, Edif. MMAyA piso 6 La Paz, Murillo Bolivie,Tel: +591-2 2148233,Email_id: adquisiciones.ucp.paap@gmail.com",None,Engg,Anand,2018-08-07,2018-07-12,26 days,Original Document
2,"Prequalification are invited for Consultants to Realize the feasibility Study of the Program of Development of Community Projects in the Agro-pastoral Sectors and Halieutics in the Context of Cameroon-israel Cooperation.,Consistency of the services The aim of this ASMI is to preselect the participants able to provide for the targeted sectors, on the one hand, a proposal on the feasibility of the program in each of the seventeen (17) above-mentioned municipalities, at the technical, financial...",Domestic Competitive Bidding,2.Cameroon - Consultants To Realize The Feasibility Study Of The Program Of Development Of Community Projects In The Agro-pastoral Sectors And Halieutics,FEICOM,Cameroon,"Directorate-general of Feicom Tenders Service Special Fund for Intercommunity Equipment and Intervention - Special Council Support Fund for Mutual Assistance 381, Rue 4561, Mimboman, Yaoundé 4th Bp718 Yaoundé, Cameroon,Tel: +237- 222 23 51 64, 222 22 27 28,Fax: +237-222 23 17 59",None,TP,Ashish,2018-08-08,2018-07-12,27 days,Original Document
3,Tenders are invited for Parts and Accessories of Railway Control and Signaling System,Not Specified Domestic / International Bidding ?,35.Chile - Parts And Accessories Of Railway Control And Signaling System,CORPORACIÓN NACIONAL DEL COBRE DE CHILE,Chile,"Huérfanos 1270, Santiago, Chile Casilla Postal 150-D 1,Tel: +56-2-2818 5765,Email_id: portalcompras@codelco.cl",None,BD2,Check,2018-09-08,2018-07-12,58 days,Original Document
4,"Tenders are invited for Recruitment of a Study Office to Carry Out Technical Studies on the Sections of the Axes Retained in the Bongandanga Territorial Mechanized Model Rehabilitation in the Province of Mongala,Description: On the basis of the results and achievements of the initial phase of the Agricultural Sector Rehabilitation and Recovery Support Project, the World Bank has provided the Government of the DRC with additional funding of US $ 75 million for one year. duration of two years....",Not Specified Domestic / Internation

In [31]:
data4 = data1[data1.BU == " "]
data4.sort_values(["Project Location","BU"], inplace=True)
final2=data4
# final2.drop(['text_stem','Project Title'], axis=1, inplace=True)
final2=data4[~data4["Project Location"].isin(nreg)]
final2 = final2.reset_index(drop=True)


In [32]:
dataA = final[final.BD == "Ashish"].reset_index(drop=True)
dataK = final[final.BD == "Anand"].reset_index(drop=True)
dataS = final[final.BD == "Saumya"].reset_index(drop=True)
dataG = final[final.BD == "Gurpreet"].reset_index(drop=True)
dataP = final[final.BD == "Sudeep"].reset_index(drop=True)
dataB=  final[final.BD == "All"].reset_index(drop=True)
dataC=  final[final.BD == "Check"].reset_index(drop=True)

In [33]:
from pandas import ExcelWriter
writer = ExcelWriter('OppurtinityList.xlsx')
final.to_excel(writer,'Oppurtunity')
final2.to_excel(writer,'Not_Oppurtinity')
dataA.to_excel(writer,'Ashish')
dataK.to_excel(writer,'Anand')
dataS.to_excel(writer,'Saumya')
dataG.to_excel(writer,'Gurpreet')
dataP.to_excel(writer,'Sudeep')
dataB.to_excel(writer,'All')
dataC.to_excel(writer,'Check')
writer.save()

In [34]:
# from pandas import ExcelWriter
# writer = ExcelWriter('OppurtinityList.xlsx')
# final.to_excel(writer,'Oppurtunity')
# final2.to_excel(writer,'Not_Oppurtinity')
# writer.save()